<a href="https://colab.research.google.com/github/isaacpy-cloud/TraderWolfs/blob/main/Trading_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-binance
!pip install catboost
!pip install mpl_finance
!pip install tradingview-ta
!pip install websocket-client
!pip install finta
!pip install pandas_ta

In [2]:
url = 'https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files'
!wget $url/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget $url/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

(Reading database ... 161001 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) over (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



In [3]:
!pip install ccxt
!pip install pyt

In [4]:
import talib as ta
import pandas as pd
import numpy as np
import time
import dateparser
import pytz
import json
import datetime as dt
from datetime import datetime, timedelta
from threading import Thread
import matplotlib.pyplot as plt
from mpl_finance import candlestick_ochl
import matplotlib.dates as mdates
from matplotlib.pyplot import figure
from sklearn import preprocessing, model_selection, neighbors, svm
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from catboost import CatBoost, CatBoostClassifier, CatBoostRegressor, Pool, cv
from tqdm import tqdm as tqdm
import websocket
from binance.client import Client
from binance.enums import *
from binance.websockets import BinanceSocketManager
from time import sleep
from tradingview_ta import TA_Handler, Interval, Exchange
from tradingview_ta import Compute
from requests import get
import socket
import sys
import math
from finta import TA

Api Key, Api Secret Alanı

In [5]:
api_key = 'hXjL8ujRBXbXLNaek31Z4ysCiCmfhnbRiVdApFFt8YlwBST6hUx1gpoVC4r5xjpS'
api_secret = 'zFyTQoSDZVdRvmmFnHgjP4oaPWcXC4hhBbICaavRKY4pBDKJawWOPX7hCF9uSAMw'

Ana Çalışma Fonksiyonum

In [6]:
from binance.client import Client

class BinanceConnection:
  def __init__(self,apikey,apisecret):
    self.apikey = apikey
    self.apisecret = apisecret
    self.ClientCon = Client(self.apikey,self.apisecret)

class HistoricalKlines:
  def ImportKlines(self,Client,Symbol,Interval):
    try:
      self.Klines = Client.get_klines(symbol=Symbol, interval=Interval)
      return self.Klines
    except Exception as exp:
      print(exp.status_code, flush=True)
      print(exp.message, flush=True) 

In [7]:
class Indicator_Settings:
  def T3Tilson(self,close_array, high_array, low_array, volume_factor, t3Length):
    ema_first_input = (high_array + low_array + 2 * close_array) / 4
    e1 = ta.EMA(ema_first_input, t3Length)
    e2 = ta.EMA(e1, t3Length)
    e3 = ta.EMA(e2, t3Length)
    e4 = ta.EMA(e3, t3Length)
    e5 = ta.EMA(e4, t3Length)
    e6 = ta.EMA(e5, t3Length)
    c1 = -1 * volume_factor * volume_factor * volume_factor
    c2 = 3 * volume_factor * volume_factor + 3 * volume_factor * volume_factor * volume_factor
    c3 = -6 * volume_factor * volume_factor - 3 * volume_factor - 3 * volume_factor * volume_factor * volume_factor
    c4 = 1 + 3 * volume_factor + volume_factor * volume_factor * volume_factor + 3 * volume_factor * volume_factor
    T3 = c1 * e6 + c2 * e5 + c3 * e4 + c4 * e3
    return T3

Class'larımın İçeri Aktarılması ve Bağlantının Sağlanması

In [8]:
BinanceCon = BinanceConnection(api_key,api_secret)
Clients = BinanceCon.ClientCon
Symbol = 'TROYBNB'

Bazı Farklı Sembollerin İçe Aktarılması

In [9]:
# Bu edit sayfasıdır.

In [10]:
KlinesCon = HistoricalKlines()
Indicators = Indicator_Settings()

In [11]:
def generateSupertrend(close_array, high_array, low_array, atr_period, atr_multiplier):

    try:
        atr = ta.ATR(high_array, low_array, close_array, atr_period)
    except:
        print('exception in atr:', sys.exc_info()[0], 'pair', pair, flush=True)
        print('filename', filename, flush=True)
        return False, False

    previous_final_upperband = 0
    previous_final_lowerband = 0
    final_upperband = 0
    final_lowerband = 0
    previous_close = 0
    previous_supertrend = 0
    supertrend = []
    supertrendc = 0

    for i in range(0, len(close_array)):
        if np.isnan(close_array[i]):
            pass
        else:
            highc = high_array[i]
            lowc = low_array[i]
            atrc = atr[i]
            closec = close_array[i]

            if math.isnan(atrc):
                atrc = 0

            basic_upperband = (highc + lowc) / 2 + atr_multiplier * atrc
            basic_lowerband = (highc + lowc) / 2 - atr_multiplier * atrc

            if basic_upperband < previous_final_upperband or previous_close > previous_final_upperband:
                final_upperband = basic_upperband
            else:
                final_upperband = previous_final_upperband

            if basic_lowerband > previous_final_lowerband or previous_close < previous_final_lowerband:
                final_lowerband = basic_lowerband
            else:
                final_lowerband = previous_final_lowerband

            if previous_supertrend == previous_final_upperband and closec <= final_upperband:
                supertrendc = final_upperband
            else:
                if previous_supertrend == previous_final_upperband and closec >= final_upperband:
                    supertrendc = final_lowerband
                else:
                    if previous_supertrend == previous_final_lowerband and closec >= final_lowerband:
                        supertrendc = final_lowerband
                    elif previous_supertrend == previous_final_lowerband and closec <= final_lowerband:
                        supertrendc = final_upperband

            supertrend.append(supertrendc)

            previous_close = closec

            previous_final_upperband = final_upperband

            previous_final_lowerband = final_lowerband

            previous_supertrend = supertrendc

    return supertrend

In [31]:
List_mycoin = [
"FET",
"VET",
"SKL",
"BAND",
"SUSHI",
"FIRO",
"ROSE",
"TVK",
"CHZ",
"CHR",
"DENT",
"HOT",
"DREP",
"WTC",
"BLZ",
"LTO",
"SRM",
"QLC",
"TOMO",
"COTI",
"ANKR",
"ETH",
"MATIC",
"ADA",
"XEM",
"ENJ",
"LUNA",
"XRP",
"TRX",
"DOT",
"COMP",
"FIL",
"KAVA",
"LUNA",
"PNT",
"QTUM",
"RVN",
"SOL",
"SXP",
"THETA",
"XEM",
"ZIL"
]
len(List_mycoin)

42

In [13]:
from datetime import datetime, timezone
import pyt

In [14]:
u = datetime.utcnow()
u = u.replace(tzinfo=pytz.utc) #NOTE: it works only with a fixed utc offset
Last_day = u.astimezone(pytz.timezone("Europe/Istanbul"))
print(Last_day)

2021-03-13 19:03:03.343734+03:00


In [114]:
def Start_Websocket(ws,message):
  # Datamın Alınması
  candle = json.loads(message)
  alt_candle = candle['k']
  candle_closed = alt_candle['x']

  u = datetime.utcnow()
  u = u.replace(tzinfo=pytz.utc) #NOTE: it works only with a fixed utc offset
  Last_day = u.astimezone(pytz.timezone("Europe/Istanbul"))

  if candle_closed:
    for trade_coin in List_mycoin:
      try:
        Obj_Coin = trade_coin + "USDT"
        Historic_Klines = KlinesCon.ImportKlines(Clients,Obj_Coin,Clients.KLINE_INTERVAL_15MINUTE)
        print('================================')
        print('')
        print(f"Coin Bilgileri Taranıyor : {Obj_Coin}")
        print(f"Coin Bilgileri Taranıyor - Zamanı : {Last_day}")
        print('================================')
        print('')


        # Datanın İçerisinden Bilgilerin Alınması
        open_time = [int(entry[0]) for entry in Historic_Klines]
        open = [float(entry[1]) for entry in Historic_Klines]
        high = [float(entry[2]) for entry in Historic_Klines]
        low = [float(entry[3]) for entry in Historic_Klines]
        close = [float(entry[4]) for entry in Historic_Klines]

          #Son Değerlerin Alınması
        last_closing_price = close[-1]
        previous_closing_price = close[-2]
        close_array = np.asarray(close)
        close_finished = close_array[:-1]

        high_array = np.asarray(high)
        low_array = np.asarray(low)
        close_array = np.asarray(close)

        new_time = [datetime.fromtimestamp(time / 100) for time in open_time]
        new_time_x = [date.strftime("%y-%m-%d") for date in new_time]
        volume_factor = 0.7
        supertrend = generateSupertrend(close_array, high_array, low_array, atr_period=10, atr_multiplier=2)



        son_kapanis = close_array[-1]
        onceki_kapanis = close_array[-2]
        son_supertrend_deger = supertrend[-1]
        onceki_supertrend_deger = supertrend[-2]
        

          # renk yeşile dönüyor, trend yükselişe geçti
        if son_kapanis > son_supertrend_deger and onceki_kapanis < onceki_supertrend_deger:

          order_proc = order(Obj_Coin,son_kapanis)
          print('================================')
          print('')
          print(f"SuperTrend Al Sinyali Üretti Alış Değeri: {float(last_closing_price)}")
          print(f"SuperTrend Al Sinyali Üretti Zamanı: {Last_day}")
          print(f"SuperTrend Al Sinyali Üretti Coin Sembolü: {Obj_Coin}")
          print(f"SuperTrend Alışı: {order_proc}")
          print('================================')
          print('')
                  
          

          # renk kırmızıya dönüyor, trend düşüşe geçti
        if son_kapanis < son_supertrend_deger and onceki_kapanis > onceki_supertrend_deger:

          trades = Clients.get_my_trades(symbol=Obj_Coin)
          if trades == "":
            print("Son İşlem Bulunamadı")
          else:
            my_last_trades = trades[-1]
            selling_signal = satis_sinyali(last_clcosing_price,my_last_trades['price'])

          if selling_signal == 1:
            order_proc2 = sell(trade_coin,son_kapanis)
            print('================================')
            print('')
            print(f"SuperTrend Sat Sinyali Üretti Satış Değeri: {float(last_closing_price)}")
            print(f"SuperTrend Sat Sinyali Üretti Zamanı: {Last_day}")
            print(f"SuperTrend Sat Sinyali Üretti Coin Sembolü: {Obj_Coin}")
            print(f"SuperTrend Satışı: {order_proc2}")
            print('================================')
            print('')
          elif selling_signal == 0:
            print('================================')
            print('')
            print(f"SuperTrend Sat Sinyali Üretti Satış Değeri: {float(last_closing_price)}")
            print(f"SuperTrend Sat Sinyali Üretti Zamanı: {Last_day}")
            print(f"SuperTrend Sat Sinyali Üretti Coin Sembolü: {Obj_Coin}")
            print(f"SuperTrend Satışı: {order_proc2}")
            print('================================')
            print('')
          
      except:
        print(Obj_Coin)

In [115]:
trades = Clients.get_my_trades(symbol='ADABTC')
trades = trades[-1]
trades['price']

'0.00001037'

In [116]:
def satis_sinyali(closing_price,buying_price):
  sell_signal = buying_price * 3 / 100
  signal = buying_price + sell_signal
  
  if closing_price >= signal:
    return 1

  elif closing_rpice <= signal:
    return 0

In [117]:
def Open_Websocket(ws):
  ip_adress = get("https://api.ipify.org").text
  local_ip = socket.gethostbyname(socket.gethostname())
  get_adress = format(ip_adress)
  hasher_newpass = '3LywfTjcbJYvdPN4vAwCBVagYa5nfnpzBb'
  print("Web Soketlerine Bağlanıldı Ip Adresiniz: " + ip_adress)

In [118]:
def Stop_Websocket(ws):
  print("Botunuz Durdurulmuştur.")

In [119]:
ip_adress = get("https://api.ipify.org").text
local_ip = socket.gethostbyname(socket.gethostname())
get_adress = format(ip_adress)
hasher_newpass = '3LywfTjcbJYvdPN4vAwCBVagYa5nfnpzBb'
print("Web Soketlerine Bağlanıldı Ip Adresiniz: " + ip_adress)

Web Soketlerine Bağlanıldı Ip Adresiniz: 104.198.15.124


In [120]:
info = Clients.get_account()
info
new_array = []

array = info['balances']

for byte_array in array:
  if float(byte_array['free']) >= 0.00000001:
    new_array.append(byte_array['free'])


def current_balances(new_array):
  my_balance = new_array
  my_btc_balances = my_balance[0]
  return my_btc_balances

btc_balances = current_balances(new_array)
print(float(btc_balances) / 2)
two_balance = float(btc_balances) / 2
float(two_balance)

1.65e-07


1.65e-07

In [121]:
wallet_control_array_symbol = []
wallet_control_array_coin = []
wallet_control_array_locked = []

In [122]:
def cuzdan():
  wallet_control_array_symbol.clear()
  wallet_control_array_coin.clear()
  wallet_control_array_locked.clear()

  info = Clients.get_account()
  var1 = info['balances']

  for vastil in var1:
    symbolm = vastil['asset']
    coinm = vastil['free']
    lockedm = vastil['locked']
    wallet_control_array_symbol.append(symbolm)
    wallet_control_array_coin.append(coinm)
    wallet_control_array_locked.append(lockedm)

  wallet_coins = wallet_control_array_symbol.index('BTC')
  btc_contr = wallet_control_array_coin[wallet_coins]
  return btc_contr

In [123]:
def order(SYMBOL,VALUE):
  try:
    my_btc = cuzdan()

    btc_Val1 = float(my_btc)
    order_value = float(btc_Val1) / float(VALUE)
    order_value = int(order_value)
    order = Clients.order_market_buy(
    symbol=SYMBOL,
    quantity=order_value)

    
    return order
  except:
    order = "Cüzdanınızda Yeterli Birim Yok"
    return order

In [124]:
# my_btc = cuzdan()
# VALUE = "0.0001007"
# SYMBOL = "QTUMBTC"
# btc_Val1 = float(my_btc)

# order_value = float(btc_Val1) / float(VALUE)
# order_value = int(order_value)
# order = Clients.order_market_buy(
# symbol=SYMBOL,
# quantity=order_value)
# print(order)

In [125]:
# print(sell("QTUM",0.001007))

In [126]:
def sell(SYMBOL,VALUE):
  try:
    try_cuzdan = cuzdan()
    wallet_coins = wallet_control_array_symbol.index(SYMBOL)
    valsss = wallet_control_array_coin[wallet_coins]
    coin_Strip = valsss

    coinlist = list(str(coin_Strip))
    my_value = ""
    sayac = 0
    for bb in coinlist:
      my_value = my_value + bb
      sayac += 1
      if sayac > 2:
        break

    SYMBOL = SYMBOL + "BTC"
    order = Clients.order_market_sell(
    symbol=SYMBOL,
    quantity=my_value)
    return order
  except:
    order = "Cüzdanınızda Yeterli Birim Yok"
    return order

Eğer değer 0.1 > ise o zaman bu fonksiyon çalışır
LOT_SİZE = minimum gönderilen değer
Eğer değer bu sayının altındaysa hata veriyor

In [127]:
# SYMBOL = "BAND"
# if SYMBOL == "BAND":
#   try_cuzdan = cuzdan()
#   wallet_coins = wallet_control_array_symbol.index(SYMBOL)
#   valsss = wallet_control_array_coin[wallet_coins]
#   coin_Strip = 0.9990000

#   coinlist = list(str(coin_Strip))
#   my_value = ""
#   sayac = 0
#   for bb in coinlist:
#     my_value = my_value + bb
#     sayac += 1
#     if sayac > 2:
#       break

#   print(my_value)


In [ ]:
SOCKET = "wss://stream.binance.com:9443/ws/btcusdt@kline_15m"
ws = websocket.WebSocketApp(SOCKET, on_open=Open_Websocket, on_close=Stop_Websocket, on_message=Start_Websocket)
ws.run_forever()

Web Soketlerine Bağlanıldı Ip Adresiniz: 104.198.15.124

Coin Bilgileri Taranıyor : FETUSDT
Coin Bilgileri Taranıyor - Zamanı : 2021-03-13 20:15:00.066284+03:00


Coin Bilgileri Taranıyor : VETUSDT
Coin Bilgileri Taranıyor - Zamanı : 2021-03-13 20:15:00.066284+03:00


Coin Bilgileri Taranıyor : SKLUSDT
Coin Bilgileri Taranıyor - Zamanı : 2021-03-13 20:15:00.066284+03:00


Coin Bilgileri Taranıyor : BANDUSDT
Coin Bilgileri Taranıyor - Zamanı : 2021-03-13 20:15:00.066284+03:00


Coin Bilgileri Taranıyor : SUSHIUSDT
Coin Bilgileri Taranıyor - Zamanı : 2021-03-13 20:15:00.066284+03:00


Coin Bilgileri Taranıyor : FIROUSDT
Coin Bilgileri Taranıyor - Zamanı : 2021-03-13 20:15:00.066284+03:00


Coin Bilgileri Taranıyor : ROSEUSDT
Coin Bilgileri Taranıyor - Zamanı : 2021-03-13 20:15:00.066284+03:00

400
Invalid symbol.

Coin Bilgileri Taranıyor : TVKUSDT
Coin Bilgileri Taranıyor - Zamanı : 2021-03-13 20:15:00.066284+03:00

TVKUSDT

Coin Bilgileri Taranıyor : CHZUSDT
Coin Bilgileri Taranıyor - 